In [2]:
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Output
import pandas as pd
from ipyleaflet import Map
import json

In [5]:
weather = pd.read_csv("osun_weather.csv")
weather

,city,variables,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,Osogbo,sunny,9.2,4.2,1.2,0.5,0.5,0.1,0.0,0.0,0.0,0.0,0.3,4.0
1,Osogbo,cloudy,21.2,22.8,26.3,23.5,19.5,13.9,10.9,10.6,8.8,18.1,27.4,26.0
2,Osogbo,precipitation,4.1,4.7,8.0,14.9,20.8,25.1,24.6,21.8,26.7,19.8,4.7,3.8
3,Ile-Ife,sunny,7.1,3.3,1.6,1.4,0.7,0.0,0.0,0.0,0.0,0.0,0.2,2.8
4,Ile-Ife,cloudy,23.2,23.8,26.0,23.1,19.7,14.3,12.5,11.3,9.9,17.8,27.4,27.0
5,Ile-Ife,precipitation,2.9,4.1,7.8,15.8,22.1,25.6,22.5,20.3,25.6,21.5,5.5,3.4
6,Ilesa,sunny,9.2,4.2,1.2,0.5,0.5,0.1,0.0,0.0,0.0,0.0,0.3,4.0
7,Ilesa,cloudy,21.2,22.8,26.3,23.5,19.5,13.9,10.9,10.6,8.8,18.1,27.4,26.0
8,Ilesa,precipitation,4.1,4.7,8.0,14.9,20.8,25.1,24.6,21.8,26.7,19.8,4.7,3.9
9,Iwo,sunny,10.0,5.6,2.7,2.2,1.1,0.1,0.0,0.1,0.0,0.0,0.2,0.2


In [7]:
out = Output()
cds = {}
with open('latlng.json') as fp:
    cds = json.load(fp)

cds

{'All': {'lat': 7.5629, 'lng': 4.52},
 'Osogbo': {'lat': 7.7827, 'lng': 4.5418},
 'Ile-Ife': {'lat': 7.4905, 'lng': 4.5521},
 'Ilesa': {'lat': 7.6395, 'lng': 4.7588},
 'Iwo': {'lat': 7.6292, 'lng': 4.1872}}

In [9]:
with out:
    display(weather)

In [11]:
weather_cities = ["All"] + list(weather.city.unique())
variables_type = ["All"] + list(weather.variables.unique())

In [14]:
dd_city = widgets.Dropdown(options = weather_cities, description = "City")
dd_variables = widgets.Dropdown(options = variables_type, description = "Variables")
float_min_hilite = widgets.FloatText(value = 5.0, description = "Highlighter")
color_picker = widgets.ColorPicker(value = 'blue', description = "Highlight Color")

my_map = Map(center = [35.5175, -86.5804], zoom = 7)

hbox1 = widgets.HBox([dd_city, dd_variables])
hbox2 = widgets.HBox([float_min_hilite, color_picker])

vbox = widgets.VBox([hbox1, hbox2])

tab = widgets.Tab()
tab.children = [vbox, my_map]
tab.set_title(0, 'Variables')
tab.set_title(1, 'Map')

float_min_hilite.value + 0.1

5.1

In [16]:
def hilite_variables(variables):
    return "color: {0}".format(color_picker.value if variables >= float_min_hilite.value else 'black')

def query_weather(city, variables):
    out.clear_output()
    city_filter = weather if city == "All" else weather[weather.city == city]
    variables_filter = city_filter if variables == "All" else city_filter[city_filter.variables == variables]
    with out:
        display(variables_filter.style.applymap(hilite_variables, subset = variables_filter.columns[2:]))

def displayVisuals(city):
    

In [15]:
def dd_city_observer(bunch):
    cd = cds[bunch.new]
    my_map.center = [cd['lat'], cd['lng']]
    my_map.zoom = 7 if bunch.new == "All" else 10
    query_weather(bunch.new, dd_variables.value)
    
def dd_variables_observer(bunch):
    query_weather(dd_city.value, bunch.new)
    
def float_min_hilite_observer(bunch):
    query_weather(dd_city.value, dd_variables.value)

def color_picker_observer(bunch):
    query_weather(dd_city.value, dd_variables.value)

In [19]:
dd_city.observe(dd_city_observer, names = "value")
dd_variables.observe(dd_variables_observer, names = "value")
float_min_hilite.observe(float_min_hilite_observer, names = "value")
color_picker.observe(color_picker_observer, names = "value")

In [21]:
display(tab)

In [22]:
display(out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<pandas.io.formats.style.Styler at 0x1…